In [1]:
import os

# Define directory structure
dirs = [
    "data",
    "models",
    "optim",
    "loss",
    "utils",
    "experiments"
]

# Define empty files to create
files = [
    "train.py",
    "config.py",
    "requirements.txt",
    "README.md",
    "data/dataloader.py",
    "models/layers.py",
    "models/neuralnet.py",
    "optim/optimizers.py",
    "loss/loss_functions.py",
    "utils/metrics.py",
    "utils/wandb_utils.py",
    "experiments/sweep_config.yaml"
]

# Create directories
for dir_path in dirs:
    os.makedirs(dir_path, exist_ok=True)

# Create empty files
for file_path in files:
    with open(file_path, 'w') as f:
        f.write("# " + os.path.basename(file_path))

print("✅ Project structure created successfully.")


✅ Project structure created successfully.


conda command
conda create -n dl_cpu_env python=3.10 pytorch torchvision torchaudio cpuonly numpy pandas -c pytorch -c conda-forge


In [2]:
import torch
w=torch.randn((30,3))
x=torch.randn((3,))
b=torch.randn((30,))


In [6]:
x.shape,w.shape,b.shape

(torch.Size([3]), torch.Size([30, 3]), torch.Size([30]))

In [4]:
torch.equal(w@x+b,x@w.T+b)

True

In [36]:
import torch

# suppose
# A has shape (B, k2)
# B has shape (B, k1)
B = 5
k1 = 3
k2 = 4
A = torch.randn(B, k2)
B = torch.randn(B, k1)

# step 1: make them broadcastable to (B, k2, k1)
#   A.unsqueeze(2) has shape (B, k2, 1)
#   B.unsqueeze(1) has shape (B, 1, k1)
outer = A.unsqueeze(2) * B.unsqueeze(1)
# outer.shape == (B, k2, k1)

# step 2: sum over the batch dimension → (k2, k1), then add the leading dim
final = outer.sum(dim=0, keepdim=True)
# final.shape == (1, k2, k1)


In [38]:
final

tensor([[[-0.5342, -0.6138, -0.4520],
         [ 0.0584,  1.5689,  1.0740],
         [-0.1158, -0.8078,  1.9790],
         [ 0.9095, -0.0090,  4.5990]]])

In [39]:
A.T

tensor([[-0.5245, -0.4977, -0.2554, -0.2298,  0.6503],
        [ 0.0698,  0.7999, -0.1477, -0.3370, -1.6321],
        [-0.3379, -0.8551,  0.6156,  0.0560, -0.8219],
        [ 1.4062, -2.1012, -0.0517,  1.8075, -0.3695]])

In [37]:
A.T @ B

tensor([[-0.5342, -0.6138, -0.4520],
        [ 0.0584,  1.5689,  1.0740],
        [-0.1158, -0.8078,  1.9790],
        [ 0.9095, -0.0090,  4.5990]])

In [35]:
A.unsqueeze(2)

tensor([[[ 1.2372],
         [-1.0038],
         [ 0.8978],
         [ 1.2709]],

        [[-1.8119],
         [-0.1455],
         [ 0.4296],
         [ 1.7042]],

        [[ 1.1360],
         [ 0.1541],
         [ 0.0899],
         [ 2.1361]],

        [[-0.6191],
         [-0.3832],
         [-0.3599],
         [-1.0618]],

        [[ 0.3360],
         [ 1.1204],
         [-1.0190],
         [-1.2992]]])

In [34]:
B.unsqueeze(1)

tensor([[[ 0.5908, -0.6767,  1.2283]],

        [[ 0.9299,  0.1274,  0.7840]],

        [[ 1.2670,  0.5035, -0.0287]],

        [[-1.2888,  0.7492,  0.0628]],

        [[-0.5339,  1.2623,  0.5710]]])

In [7]:
final

tensor([[[ 1.1039, -0.5358,  0.2195],
         [-0.6374,  1.8656, -0.7358],
         [ 2.0516, -2.0635,  0.8326],
         [ 7.1039, -2.0029,  2.0271]]])

In [14]:
A

tensor([[ 1.2372, -1.0038,  0.8978,  1.2709],
        [-1.8119, -0.1455,  0.4296,  1.7042],
        [ 1.1360,  0.1541,  0.0899,  2.1361],
        [-0.6191, -0.3832, -0.3599, -1.0618],
        [ 0.3360,  1.1204, -1.0190, -1.2992]])

In [17]:
B[0,:]

tensor([ 0.5908, -0.6767,  1.2283])

In [18]:
A[0,:].T

tensor([ 1.2372, -1.0038,  0.8978,  1.2709])

In [23]:
A[0,:].T.unsqueeze(0).shape

torch.Size([1, 4])

In [16]:
A[0,:].T@B[0,:]

/tmp/ipykernel_48265/2399136939.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647176142/work/aten/src/ATen/native/TensorShape.cpp:3683.)
  A[0,:].T@B[0,:]


RuntimeError: inconsistent tensor size, expected tensor [4] and src [3] to have the same number of elements, but got 4 and 3 elements respectively

In [30]:
A


tensor([[ 1.2372, -1.0038,  0.8978,  1.2709],
        [-1.8119, -0.1455,  0.4296,  1.7042],
        [ 1.1360,  0.1541,  0.0899,  2.1361],
        [-0.6191, -0.3832, -0.3599, -1.0618],
        [ 0.3360,  1.1204, -1.0190, -1.2992]])

In [32]:
A[0,:].T.unsqueeze(1)

tensor([[ 1.2372],
        [-1.0038],
        [ 0.8978],
        [ 1.2709]])

In [33]:
B[0,:].unsqueeze(0)

tensor([[ 0.5908, -0.6767,  1.2283]])

In [31]:
B

tensor([[ 0.5908, -0.6767,  1.2283],
        [ 0.9299,  0.1274,  0.7840],
        [ 1.2670,  0.5035, -0.0287],
        [-1.2888,  0.7492,  0.0628],
        [-0.5339,  1.2623,  0.5710]])

In [29]:
A[0,:].T.unsqueeze(1)@B[0,:].unsqueeze(0)

tensor([[ 0.7309, -0.8372,  1.5196],
        [-0.5930,  0.6793, -1.2330],
        [ 0.5304, -0.6076,  1.1028],
        [ 0.7508, -0.8600,  1.5610]])

In [28]:
outer

tensor([[[ 7.3086e-01, -8.3722e-01,  1.5196e+00],
         [-5.9302e-01,  6.7932e-01, -1.2330e+00],
         [ 5.3041e-01, -6.0759e-01,  1.1028e+00],
         [ 7.5078e-01, -8.6004e-01,  1.5610e+00]],

        [[-1.6848e+00, -2.3081e-01, -1.4205e+00],
         [-1.3529e-01, -1.8534e-02, -1.1407e-01],
         [ 3.9946e-01,  5.4724e-02,  3.3679e-01],
         [ 1.5846e+00,  2.1709e-01,  1.3360e+00]],

        [[ 1.4393e+00,  5.7196e-01, -3.2643e-02],
         [ 1.9528e-01,  7.7606e-02, -4.4291e-03],
         [ 1.1384e-01,  4.5240e-02, -2.5819e-03],
         [ 2.7063e+00,  1.0755e+00, -6.1381e-02]],

        [[ 7.9797e-01, -4.6386e-01, -3.8889e-02],
         [ 4.9387e-01, -2.8708e-01, -2.4069e-02],
         [ 4.6385e-01, -2.6963e-01, -2.2605e-02],
         [ 1.3685e+00, -7.9551e-01, -6.6694e-02]],

        [[-1.7941e-01,  4.2417e-01,  1.9188e-01],
         [-5.9820e-01,  1.4143e+00,  6.3979e-01],
         [ 5.4403e-01, -1.2862e+00, -5.8185e-01],
         [ 6.9364e-01, -1.6400e+00, -7.418

In [9]:
final

tensor([[[ 1.1039, -0.5358,  0.2195],
         [-0.6374,  1.8656, -0.7358],
         [ 2.0516, -2.0635,  0.8326],
         [ 7.1039, -2.0029,  2.0271]]])

In [8]:
import torch
# B = 5
# k1 = 3
# k2 = 4
# A = torch.randn(B, k2)
# B = torch.randn(B, k1)

# direct outer‐product per batch
outer = torch.einsum('bi,bj->bij', A, B)
# outer.shape == (B, k2, k1)

# sum over batch
final = outer.sum(0, keepdim=True)
# final.shape == (1, k2, k1)


In [43]:
k=range(5,0,-1)
print(type(k))

<class 'range'>


In [45]:
for i in range(5,-1,-1):
    print(i)

5
4
3
2
1
0
